In [25]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg, add the modules
import operator # sorting
from math import *
import matplotlib.pyplot as plt

sys.path.append('../')

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

import prep
from warp import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
trace_file = 'trace.csv'
metrics_file = 'metrics.csv'

# read trace and metrics into dataframe, from generated trace file
df_kernel_trace, df_metrics = prep.Prep_trace_metrics(trace_file, metrics_file)

In [27]:
unique_kerns = []
for index, kenname in enumerate(df_kernel_trace.Name.unique()):
    if index > 0 and 'memcpy' not in kenname:
        head = kenname.split('(')[0]
        if '<' in head:
            head = head.split('<')[0]
        #print head
    
        if 'void' in head:
            head = head.split(' ')
            #print head[1]
            head = head[1]
            
        if head not in unique_kerns:
            unique_kerns.append(head)
            print head

vectorAdd


In [28]:
target_kern_name = 'vectorAdd'
sass_result = 'vectorAdd.sm_52.sass.result'
kernel_stat_out ='kernelStat_' + target_kern_name + '.csv'

out_columns = prep.init_df_columns(df_metrics)

df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
    
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

#df_kern.to_csv('kernel_stat.csv', index=False, encoding='utf-8')

warp_inst = WarpInst()

warp_inst.run(df_kern, sass_result)

LDG (global load) clocks (per warp) 		 = 650.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 0.0
STS (shared memory store) clocks (per warp) 	 = 0.0
=> Memory inst. (per warp) 			 = 669.0 (clocks)

Integer inst. (per warp) 		 = 180.0 (clocks)
FP32 inst. (per warp) 			 = 15.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 45.0 (clocks)
=> Compute inst. (per warp) 		 = 240.0 (clocks)

Memory Intensive : mem / cmp = 2.7875


In [29]:
df_kern.insert(1, 'm2c_ratio', warp_inst.mem_clks / warp_inst.cmp_clks)

df_kern.to_csv(kernel_stat_out, index=False, encoding='utf-8')

In [30]:
df_kern

,kern_name,m2c_ratio,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,vectoradd,2.7875,0.00448,782.0,1.0,1.0,64.0,1.0,1.0,8.0,...,Low (3),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.568365,0.403547%,0.000000%
